# Big Data Modeling and Management Assigment


## 🍺 The Beer project  🍺

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_

#### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions

#### Connection details to the neo4j database
```
Host: rhea.isegi.unl.pt:7474  
Username: neo4j  
Password: F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ
Connect URL : bolt://rhea.isegi.unl.pt:7687
```


##### Submission      

Submission of the query results to be done to the group's redis database (explained on the first class, credentials sent via email).  
The following format is expected:
```
    >>> redis.set("0", "358873")
```

This result should be the anwser of the group to question 0

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of May 2nd**


**Note:**
Remember the Neo4j is a shared database and when creating in-memory graphs please use your group's prefix.  
Ex. Instead of `my-graph` as the name of your graph please use `group0-my-graph`.

In [ ]:
pip install redis

     |████████████████████████████████| 81kB 4.0MB/s 


In [ ]:
import redis

r = redis.Redis(
    host='rhea.isegi.unl.pt',                            # Url where the database is hosted
    port='6413',                                         # Server port in which redis is running
    password='DA6AfHBR1qGqN9eyrvWjoaQmYsV8EXoo', # Password to authenticate on the server
    decode_responses=True
)

server_info = r.info()

print("Server version "+ server_info["redis_version"])


Server version 6.2.2


In [ ]:
!pip install py2neo
#!pip install -U ipython
import py2neo
from pprint import pprint
from py2neo import GraphService

     |████████████████████████████████| 204kB 6.1MB/s 
     |████████████████████████████████| 348kB 18.1MB/s 
     |████████████████████████████████| 153kB 17.7MB/s 
     |████████████████████████████████| 3.2MB 20.8MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel for neotime: filename=neotime-1.7.4-cp37-none-any.whl size=20541 sha256=b566572743f2d941f070b63f92a4bcb4bc58c30df5940ff702c79b3caf26757c
  Stored in directory: /root/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built neotime
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.10 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18


In [ ]:
host = 'rhea.isegi.unl.pt'
username = 'neo4j'
password = 'F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ'
port  = '7474'

secure_graph = py2neo.Graph(f"http://{username}:{password}@{host}:{port}")
secure_graph.run("MATCH () RETURN count(*)").data()

[{'count(*)': 9647598}]

1. How many different countries exist in the database?

In [ ]:
result = secure_graph.run("""
        MATCH (c:Country)
        RETURN count(c)
""").data()
pprint(result)

[{'count(c)': 200}]


In [ ]:
r.set("1", "200")

True

2. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers?
    1. Which `Country` has the most reviews for its beers?

In [ ]:
#A
result = secure_graph.run("""
        MATCH (r:Reviews)-[a:ABOUT]->(b:Beers)
        RETURN b.name, count(r)
        order by count(r) desc
        limit 1
""").data()
pprint(result)

[{'b.name': 'IPA', 'count(r)': 31387}]


In [ ]:
#B
result = secure_graph.run("""
        MATCH(r:Reviews)-[:ABOUT]->(b:Beers)-[:BREWED_AT]->(br:Breweries)
        RETURN br.name, count(r)
        order by count(r) desc
        LIMIT 1
""").data()
pprint(result)

[{'br.name': 'Sierra Nevada Brewing Co.', 'count(r)': 175161}]


In [ ]:
#C
result = secure_graph.run("""
        MATCH(r:Reviews)-[:ABOUT]->(b:Beers)-[:BREWED_AT]->(br:Breweries)-[:FROM]->(c:Country)
        RETURN c.country_digit, count(r)
        order by count(r) desc
        LIMIT 1
""").data()
pprint(result)

[{'c.country_digit': 'US', 'count(r)': 7524410}]


In [ ]:
r.set("2.1", "IPA")

True

In [ ]:
r.set("2.2", "Sierra Nevada Brewing Co.")

True

In [ ]:
r.set("2.3", 'US')

True

3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman? ELEONORA

In [ ]:
#3
secure_graph.run("""
    MATCH (u:Username)-[:MADE]-(r:Reviews)-[:ABOUT]-(b:Beers)-[]-()-[]-(:Username{user_name:'CTJman'})
    RETURN u, count(r)
    ORDER BY count(r) DESC
    LIMIT 3
""").data()
#in the count we considered the number of reviews

[{'count(r)': 1428, 'u': Node('Username', user_name='acurtis')},
 {'count(r)': 1257, 'u': Node('Username', user_name='Texasfan549')},
 {'count(r)': 1205, 'u': Node('Username', user_name='kjkinsey')}]

In [ ]:
r.set("3", 'acurtis, Texasfan549, kjkinsey')

True


4. Which Portuguese brewery has the most beers?

In [ ]:
#4
result = secure_graph.run("""
        MATCH (c:Country)<-[:FROM]-(br:Breweries)<-[:BREWED_AT]-(b:Beers)
        WHERE c.country_digit = 'PT'
        RETURN br.name, count(b)
        order by count(b) desc
        LIMIT 1
""").data()
pprint(result)

[{'br.name': 'Dois Corvos Cervejeira', 'count(b)': 40}]


In [ ]:
r.set("4","Dois Corvos Cervejeira")

True

5. From those beers (the ones returned from the previous question), which has the most reviews?

In [ ]:
#5
result = secure_graph.run("""
        MATCH (c:Country)<-[:FROM]-(br:Breweries)<-[:BREWED_AT]-(b:Beers)<-[:ABOUT]-(r:Reviews)
        WHERE br.name = 'Dois Corvos Cervejeira'
        RETURN b.name, count(r)
        order by count(r) desc
        LIMIT 1
""").data()
pprint(result)

[{'b.name': 'Finisterra', 'count(r)': 10}]


In [ ]:
r.set("5","Finisterra")

True

6. On average how many different beer styles does each brewery produce? TOMAS

In [ ]:
#6
result = secure_graph.run("""
        match (b:Beers)-[:OF_TYPE]->(s:Style)
        with distinct count(s) as nr_styles
        match (br:Breweries)
        with (nr_styles*1.0/count(br)*1.0) as AVG_Beer_Styles
        return AVG_Beer_Styles
""").data()
pprint(result)

[{'AVG_Beer_Styles': 7.127991737342841}]


In [ ]:
r.set("6","7.128")

True

7. Which brewery produces the strongest beers according to ABV?

In [ ]:
result = secure_graph.run("""
        MATCH   (b:Beers) - [:BREWED_AT] -> (br: Breweries)
        WHERE b.abv <> 'Unknown'
        RETURN  b.name AS beer ,  b.abv, br.name AS brewery
        ORDER BY b.abv DESC
        LIMIT 1
""").data()
pprint(result)

[{'b.abv': '9.99',
  'beer': 'Imperial Cascadian',
  'brewery': 'New Belgium Brewing Company'}]


In [ ]:
r.set("7", 'New Belgium Brewing Company')

True

8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [ ]:
#we want to see for look and smell which are the best beers, and we ordered them descending
#being mindful to take out unkown values

result = secure_graph.run("""
        MATCH   (rw:Reviews) - [:ABOUT] -> (b: Beers) - [:OF_TYPE] -> (s:Style)
        WHERE rw.look <> 'Unknown' and rw.smell <> 'Unknown'
        RETURN  s.name
        ORDER BY rw.look DESC, rw.smell DESC
        LIMIT 1
""").data()
pprint(result)

[{'s.name': 'English Oatmeal Stout'}]


In [ ]:
r.set("8","English Oatmeal Stout")

True

9. Using Graph Algorithms answer **two** of the following questions:\
    A. Which two Countries are most similiar when it comes to their **top 10** most produced Beer styles?\
    B. Which beer has the most similar reviews as the beer `Super Bock Stout`?\
    C. Which user is the most influential when it comes to reviews made?\
    D. Which beer styles are more central when it comes the amount of beers? \

In [ ]:
#A query.
#for the most similar countries we wanted to get the top 10 beers for every country and compare the couple of countries.

result = secure_graph.run("""
MATCH (b:Beers)-[]->(br:Breweries)-[]-(c:Country)
    WITH b as beer, c as country, count(*) as times
    order by times desc
    WITH country, collect(beer)[..10] as top_10
    UNWIND top_10 as b_top_10
    RETURN country.country_digit as source, id(b_top_10) as target
    limit 20
""").data()
pprint(result)

[{'source': 'BE', 'target': 322527},
 {'source': 'BE', 'target': 263976},
 {'source': 'BE', 'target': 190945},
 {'source': 'BE', 'target': 182407},
 {'source': 'BE', 'target': 169343},
 {'source': 'BE', 'target': 165964},
 {'source': 'BE', 'target': 134554},
 {'source': 'BE', 'target': 460046},
 {'source': 'BE', 'target': 394042},
 {'source': 'BE', 'target': 390226},
 {'source': 'US', 'target': 375808},
 {'source': 'US', 'target': 259713},
 {'source': 'US', 'target': 299834},
 {'source': 'US', 'target': 260497},
 {'source': 'US', 'target': 206847},
 {'source': 'US', 'target': 203148},
 {'source': 'US', 'target': 197681},
 {'source': 'US', 'target': 180565},
 {'source': 'US', 'target': 163309},
 {'source': 'US', 'target': 162566}]


In [ ]:
#A create node
#when creating the graph unfortunately there is a "type" error which we couldn't solve.
#apart from that we are confident this is the right code

secure_graph.run(f"""
    CALL gds.graph.create.cypher(
    'group21_graph',
    'MATCH (b:Beers) return id(b) as id UNION ALL MATCH (c:Country) return id(c) as id',
    'MATCH (b:Beers)-[]-(br:Breweries)-[]-(c:Country)
    WITH b as beer, c as country, count(*) as times
    order by times desc
    WITH country, collect(beer)[..10] as top_10
    UNWIND top_10 as b_top_10
    RETURN c.country_digit as source, id(b_top_10) as target'
    )
""").data()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: ignored

In [ ]:
r.set("9A","BE, US")
#this answer is based on the query only, we are not sure it is the correct one as we couldn't run the graph part.

True

In [ ]:
#B
#we didn't submit this question, but we tried anyway to answer it. the runtime crushes and we never managed to see the results.
result = secure_graph.run("""
match (b1:Beers)<-[:ABOUT]-(r1:Reviews)
with r1 as w
where b1.name = 'Super Bock Stout'
match (b:Beers)<-[:ABOUT]-(r:Reviews)
RETURN gds.alpha.similarity.jaccard([toFloat(w.smell),toFloat(w.taste),toFloat(w.look)],[toFloat(r.smell),toFloat(r.taste),toFloat(r.look)]) AS similarity,b.name
order by similarity desc
limit 1
""").data()
result

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyboardInterrupt: ignored

In [ ]:
#D
#The question was understood as which is the style that contains the most amount of beers, translated in
#degree centrality algorithm (max number of connections between beers and styles)

In [ ]:
result = secure_graph.run("""
 MATCH (st:Style)<-[:OF_TYPE]-(b:Beers)
 RETURN st.name, COUNT(b.name) AS beers
 ORDER BY beers DESC LIMIT 4
""").data()
result
#this answer was done without the help of graph algorithms.

[{'beers': 44719, 'st.name': 'American IPA'},
 {'beers': 22159, 'st.name': 'American Pale Ale (APA)'},
 {'beers': 18338, 'st.name': 'American Imperial IPA'},
 {'beers': 18167, 'st.name': 'Belgian Saison'}]

In [ ]:
#When creating the cypher projection something goes wrong as the relationshipcounts is always zero.
secure_graph.run("""
    CALL gds.graph.create.cypher(
        'Group21popularbeers15',
        'MATCH (n:Style) RETURN id(n) AS id',
        'MATCH (n:Style)-[]-(b:Beers) RETURN id(n) AS source, id(b) AS target',
        {validateRelationships:false}
    )
""").data()

[{'createMillis': 354,
  'graphName': 'Group21popularbeers15',
  'nodeCount': 113,
  'nodeQuery': 'MATCH (n:Style) RETURN id(n) AS id',
  'relationshipCount': 0,
  'relationshipQuery': 'MATCH (n:Style)-[]-(b:Beers) RETURN id(n) AS source, id(b) AS target'}]

In [ ]:
#given the relationshipcount 0, we couldn't manage to apply the centrality algorithm.
#we left the formula here even though the results are wrong
secure_graph.run("""
 CALL gds.alpha.degree.stream('Group21popularbeers12')
 YIELD nodeId, score
 RETURN gds.util.asNode(nodeId).name AS name, score AS beers
 ORDER BY beers DESC
 LIMIT 5
""").data()

[{'beers': 0.0, 'name': 'Russian Imperial Stout'},
 {'beers': 0.0, 'name': 'Scottish Ale'},
 {'beers': 0.0, 'name': 'American Imperial Stout'},
 {'beers': 0.0, 'name': 'American Cream Ale'},
 {'beers': 0.0, 'name': 'English Oatmeal Stout'}]

In [ ]:
#we also tried another approach in order to find the solution, but the problem is always at the relationshipcount level.
secure_graph.run("""
 CALL gds.alpha.degree.stream({
 nodeProjection: "Style",
 relationshipProjection: 'OF_TYPE'
 })
 YIELD nodeId, score
 RETURN gds.util.asNode(nodeId).name AS name, score AS beers
 ORDER BY beers DESC
 LIMIT 5
""").data()

[{'beers': 0.0, 'name': 'Russian Imperial Stout'},
 {'beers': 0.0, 'name': 'Scottish Ale'},
 {'beers': 0.0, 'name': 'American Imperial Stout'},
 {'beers': 0.0, 'name': 'American Cream Ale'},
 {'beers': 0.0, 'name': 'English Oatmeal Stout'}]

In [ ]:
r.set("9D","American IPA, American Pale Ale (APA)")
#this solution is given based on the first query done without the help of the centrality algorithm

True

10. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [ ]:
#10

#Once there were way too many beers with score and overall of 5 (maximum), we decided that the field to tiebrake this
#impass would be the number of reviews that is proportional to the number of sold units. We considered the score, overall
#and number of sold units as the fields that matter to decide which beers to recommend


result = secure_graph.run("""
        MATCH (rw:Reviews) - [:ABOUT] -> (b:Beers)
        Where rw.score = '5' and rw.overall = '5'
        return b.name, count(rw)
        ORDER BY count(rw) DESC
        limit 3
""").data()
pprint(result)

[{'b.name': 'Heady Topper', 'count(rw)': 1024},
 {'b.name': 'Pliny The Elder', 'count(rw)': 727},
 {'b.name': 'KBS (Kentucky Breakfast Stout)', 'count(rw)': 569}]


In [ ]:
r.set("10", "Heady Topper, Pliny The Elder, KBS (Kentucky Breakfast Stout)")

True